Lo primero que hice fue importar las librerías que iba a utilizar en el programa, en este caso importe *json* para poder leer el archivo provisto y *pandas* para facilitar el manejo de los datos.

In [1]:
import json
import pandas as pd

Lo siguiente fue crear variables globales que usaría para guardar las 3 collecciones de datos

In [2]:
df_pasajeros = pd.DataFrame()
df_vuelos = pd.DataFrame()
df_linea_aerea = pd.DataFrame()

Después de eso abrí el archivo json, lo abrí de esta manera ya que hasta donde sé esta es la forma correcta de abrir un archivo en python ya que previene que las variables existan más de lo que necesitan así como manejar de mejor manera el cerrar el archivo y posibles errores al leer. Dentro de este bloque fue donde fui convirtiendo los datos a dataframes de pandas y haciendoles modificaciones.

In [4]:
with open("datos_deacero.json") as json_file:
  # Lo primero fue cargar el archivo json en un diccionario
  json_data = json.load(json_file)

  # Después leí los pasajeros del 2016 en un dataframe y configure el ID del
  # pasajero como el índice de la tabla
  df_pasajeros_2016 = pd.DataFrame(json_data["CasoPractico_Pasajeros_2016"])
  df_pasajeros_2016.set_index("ID_Pasajero", inplace=True)

  # Luego hice lo mismo con los pasajeros del 2017
  df_pasajeros_2017 = pd.DataFrame(json_data["CasoPractico_Pasajeros_2017"])
  df_pasajeros_2017.set_index("ID_Pasajero", inplace=True)

  # Después hice una unión de las dos tablas, aquí fue donde me encontre el
  # primer error, el cual era había varias entradas con el mismo ID pero con
  # datos diferentes, la solución que me pareció más adecuada para este caso fue
  # borrar todas las entradas con ID repetidos ya que no tengo forma de saber
  # cuales son los datos correctos de esos IDs.
  df_pasajeros = pd.concat([df_pasajeros_2016, df_pasajeros_2017])
  df_pasajeros = df_pasajeros[~df_pasajeros.index.duplicated(keep=False)]

  # Luego carge en un dataframe los vuelos del 2016
  df_vuelos_2016 = pd.DataFrame(json_data["CasoPractico_Vuelos_2016"])

  # Luego los del 2017
  df_vuelos_2017 = pd.DataFrame(json_data["CasoPractico_Vuelos_2017"])

  # Luego uní ambos, aquí no existio el mismo problema ya que los vuelos no
  # tienen un ID como los pasajeros, por lo que no puede presentarse el mismo
  # problema, aún así ejecute el comando para eliminar duplicados en caso de
  # que hubiera líneas donde toda la columna estuviese duplicada
  df_vuelos = pd.concat([df_vuelos_2016, df_vuelos_2017], ignore_index=True)
  df_vuelos.drop_duplicates(inplace=True)

  # Finalmente carge los datos de las líneas aéreas, debido a que eran pocas no
  # ejecute el comando para eliminar duplicados pero si hubiesen sido más habría
  # realizado el mismo proceso que con los pasajeros ya que las líneas aéreas
  # sí tienen una clave única
  df_linea_aerea = pd.DataFrame(json_data["LineaAerea"])
  df_linea_aerea.set_index("Code", inplace=True)

Después de eso fue combinar estos diferentes dataframes de forma que hablaran del mismo registro, aquí tuve una gran ventaja al usar pandas ya que provee una forma bastante simple de unir dataframes, el método *join*, el cual funciona como el *join* de una base de datos, para unir los vuelos con la información de cada pasajero me pareció más adecuado usar un *inner join* ya que había IDs de pasajeros que no iban a tener información por la decisión anterior de eliminar duplicados.

In [5]:
df_full = df_vuelos.join(df_pasajeros, on="Cve_Cliente", how="inner")

Lo siguiente fue teniendo este dataframe ya con información de los vuelos y de los pasajero unirlo con las líneas aéreas, con la diferencia de que ahora fue con un *left join* en lugar de un *inner join* ya que es mejor tener algunos vuelos donde no se sabe el nombre de la aerolínea a perder esos registros.

In [6]:
df_full = df_full.join(df_linea_aerea, on="Cve_LA")

El siguiente comando fue solo para reiniciar el índice que pandas pone de forma automática para que sea secuencial, pero realmente no tiene un impacto en los datos.

In [7]:
df_full.reset_index(inplace=True)

Luego sigue uno de los requisitos del ejercicio, que para aquellos vuelos donde no se sabe el nombre de la aerolínea se despliegue "Otra", esto igual es bastante directo usando los métodos de pandas.

In [8]:
df_full["Linea_Aerea"].fillna("Otra", inplace=True)

Después seguia hacer un despliegue con solo algunas columnas, para eso lo guarde en otro dataframe pero realmente pude haber hecho un *print* a la expresión de la primera línea.

In [9]:
df_display = df_full[["Viaje", "Clase", "Precio", "Ruta", "Edad", "Linea_Aerea"]]
df_display

,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea
0,9/10/2016,Economy,60,DAL-MDW,34,Southwest
1,11/18/2016,Business,81,DAL-PHX,34,Southwest
2,6/11/2016,Economy,150,DAL-SLC,25,American Airlines
3,9/21/2016,Economy,68,DAL-AMA,25,Otra
4,7/13/2016,First Class,233,DAL-HOU,25,Aeromexico
...,...,...,...,...,...,...
207,7/28/2017,Business,142,DAL-LGA,21,Southwest
208,1/7/2017,First Class,178,DAL-AMA,21,Avianca
209,9/7/2017,First Class,163,DAL-SLC,66,Avianca
210,11/15/2017,Economy,150,DAL-OKC,66,Avianca


Lo siguiente era poder separar por semestre así como poder agrupar por varias columnas, para agrupar la única columna que iba a presentar un problema era la de "Viaje" ya que era la fecha completa pero se debe agrupar por año.
Por eso lo primero fue hacer una copia del dataframe que use para desplegar.

In [10]:
df_avg_price = df_display.copy()

Después de eso hice un casteo de la columna "Viaje" a *string* y le hice un *split* usando como separador los diagonales.

In [11]:
df_avg_price = df_avg_price.astype({"Viaje": "string"})
df_avg_price["Viaje"] = df_avg_price["Viaje"].str.split("/")

Después de haber hecho esa separación hice dos columnas extra donde puse los valores relevantes, siendo el año y el semestre, para el semestre use una función lambda ya que la lógica era muy corta para hacerlo una función separada.

In [12]:
df_avg_price["Año"] = df_avg_price["Viaje"].map(lambda x : x[2])
df_avg_price["Semestre"] = df_avg_price["Viaje"].map(lambda x : 1 if int(x[0]) <= 6 else 2)

Lo siguiente fue crear un dataframe con los datos del primer semestre, luego seleccionar solo las columnas relevantes y finalmente agrupar

In [13]:
df_first_semester = df_avg_price.query("Semestre == 1")
df_first_semester = df_first_semester[["Año", "Clase", "Ruta", "Linea_Aerea", "Precio"]]
df_first_semester.groupby(["Año", "Clase", "Ruta", "Linea_Aerea"]).mean()

Precio
Año  Clase       Ruta    Linea_Aerea              
2016 Business    DAL-AMA American Airlines    58.0
                         Avianca             132.0
                         KLM                 184.0
                 DAL-ATL Aeromexico          218.0
                         American Airlines   102.0
...                                            ...
     First Class DAL-TUL Southwest           103.0
2017 Business    DAL-AMA Aeromexico          194.0
                 DAL-ELP Southwest            93.0
     First Class DAL-AMA Avianca             178.0
                 DAL-ATL Southwest           237.0

[101 rows x 1 columns]

Luego lo mismo pero para el segundo semestre

In [15]:
df_second_semester = df_avg_price.query("Semestre == 2")
df_second_semester = df_second_semester[["Año", "Clase", "Ruta", "Linea_Aerea", "Precio"]]
df_second_semester.groupby(["Año", "Clase", "Ruta", "Linea_Aerea"]).mean()

Precio
Año  Clase       Ruta    Linea_Aerea              
2016 Business    DAL-AMA American Airlines   186.0
                 DAL-ATL American Airlines   135.0
                         Avianca             107.0
                         KLM                 227.0
                         Southwest           185.0
...                                            ...
2017 Economy     DAL-AMA Aeromexico           56.0
                 DAL-OKC Avianca             150.0
                 DAL-TUL Avianca              70.0
     First Class DAL-MDW KLM                 167.0
                 DAL-SLC Avianca             163.0

[84 rows x 1 columns]

---